In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error
# import category_encoders as ce
from sklearn.ensemble import GradientBoostingRegressor

In [60]:
macro_data = pd.read_csv('macro_data.csv')
forecast_data = pd.read_csv('forecast_starting_data.csv')

In [6]:
trn = pd.read_csv('training_data.csv')
trn.head()

/var/folders/5l/9xf1lsbn6y12fxxsh78nmsr80000gn/T/ipykernel_1752/176149714.py:1: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  trn = pd.read_csv('training_data.csv')


,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,charge_off_reason_code,mob,open_closed_flag,...,due_balance_2,due_balance_3,due_balance_4,due_balance_5,due_balance_6,due_balance_7,due_balance_8,snapshot,mth_code,industry
0,1.0,P,0,0,I 0-6,1.0,581-600,NaN,3,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201807,B
1,0.0,P,0,0,I 0-6,1.0,721-740,NaN,159,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201906,C
2,1.0,T,0,0,I 7-12,1.0,761+,NaN,22,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201905,A
3,1.0,T,0,1,I 0-6,1.0,761+,NaN,16,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201806,B
4,1.0,P,0,0,A,0.0,721-740,NaN,3,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201902,B


In [7]:
trn=trn.drop(['charge_off_reason_code'],axis=1)
trn.head()

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,mob,open_closed_flag,ever_delinquent_flg,...,due_balance_2,due_balance_3,due_balance_4,due_balance_5,due_balance_6,due_balance_7,due_balance_8,snapshot,mth_code,industry
0,1.0,P,0,0,I 0-6,1.0,581-600,3,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201807,B
1,0.0,P,0,0,I 0-6,1.0,721-740,159,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201906,C
2,1.0,T,0,0,I 7-12,1.0,761+,22,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201905,A
3,1.0,T,0,1,I 0-6,1.0,761+,16,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201806,B
4,1.0,P,0,0,A,0.0,721-740,3,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201902,B


In [8]:
trn.columns.get_loc("due_account_2")
trn.columns.get_loc("due_balance_8")

64

In [9]:
trn['overdue_months'] = 0
trn['overdue_amount'] = 0

for n in range(2, 9):
    column_name = f'due_account_{n}'
    column_name_am = f'due_balance_{n}'
    trn['overdue_months'] += n*trn[column_name]
    trn['overdue_amount'] += trn[column_name_am] 

# Check the result
print(trn[['due_account_2', 'due_account_3', 'due_account_4', 'due_account_5', 'due_account_6', 'due_account_7', 'due_account_8', 'overdue_months']])

         due_account_2  due_account_3  due_account_4  due_account_5  \
0                  0.0            0.0            0.0            0.0   
1                  0.0            0.0            0.0            0.0   
2                  0.0            0.0            0.0            0.0   
3                  1.0            0.0            0.0            0.0   
4                  0.0            0.0            0.0            0.0   
...                ...            ...            ...            ...   
5758218            0.0            0.0            0.0            0.0   
5758219            1.0            0.0            0.0            0.0   
5758220            0.0            0.0            0.0            0.0   
5758221            0.0            0.0            0.0            0.0   
5758222            0.0            0.0            0.0            0.0   

         due_account_6  due_account_7  due_account_8  overdue_months  
0                  0.0            0.0            0.0             0.0  
1    

In [10]:
trn[350:400]

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,mob,open_closed_flag,ever_delinquent_flg,...,due_balance_4,due_balance_5,due_balance_6,due_balance_7,due_balance_8,snapshot,mth_code,industry,overdue_months,overdue_amount
350,1.0,R,1,0,I 13-24,1.0,761+,22,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201908,B,0.0,0.00
351,1.0,R,0,0,A,1.0,621-640,135,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201901,B,0.0,0.00
352,0.0,P,0,0,A,0.0,721-740,21,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201807,C,0.0,0.00
353,1.0,D,0,0,A,1.0,741-760,154,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201809,C,0.0,0.00
354,1.0,I,0,0,A,1.0,761+,28,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201802,A,0.0,0.00
355,1.0,T,0,1,I 0-6,1.0,561-580,98,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201804,B,0.0,0.00
356,0.0,P,0,1,A,1.0,761+,10,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201910,A,0.0,0.00
357,1.0,I,0,0,A,1.0,761+,44,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201809,B,2.0,0.00
358,1.0,I,1,1,A,1.0,681-700,34,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201806,B,0.0,0.00
359,1.0,I,0,1,A,1.0,741-760,35,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201807,C,0.0,0.00


In [11]:
trn=trn.drop(['due_account_2'], axis=1)
trn=trn.drop(['due_account_3'], axis=1)
trn=trn.drop(['due_account_4'], axis=1)
trn=trn.drop(['due_account_5'], axis=1)
trn=trn.drop(['due_account_6'], axis=1)
trn=trn.drop(['due_account_7'], axis=1)
trn=trn.drop(['due_account_8'], axis=1)

trn[350:400]

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,mob,open_closed_flag,ever_delinquent_flg,...,due_balance_4,due_balance_5,due_balance_6,due_balance_7,due_balance_8,snapshot,mth_code,industry,overdue_months,overdue_amount
350,1.0,R,1,0,I 13-24,1.0,761+,22,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201908,B,0.0,0.00
351,1.0,R,0,0,A,1.0,621-640,135,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201901,B,0.0,0.00
352,0.0,P,0,0,A,0.0,721-740,21,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201807,C,0.0,0.00
353,1.0,D,0,0,A,1.0,741-760,154,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201809,C,0.0,0.00
354,1.0,I,0,0,A,1.0,761+,28,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201802,A,0.0,0.00
355,1.0,T,0,1,I 0-6,1.0,561-580,98,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201804,B,0.0,0.00
356,0.0,P,0,1,A,1.0,761+,10,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201910,A,0.0,0.00
357,1.0,I,0,0,A,1.0,761+,44,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201809,B,2.0,0.00
358,1.0,I,1,1,A,1.0,681-700,34,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201806,B,0.0,0.00
359,1.0,I,0,1,A,1.0,741-760,35,1.0,0.0,...,0.0,0.0,0.0,0.00,0.0,201801,201807,C,0.0,0.00


In [12]:
trn=trn.drop(['due_balance_2'], axis=1)
trn=trn.drop(['due_balance_3'], axis=1)
trn=trn.drop(['due_balance_4'], axis=1)
trn=trn.drop(['due_balance_5'], axis=1)
trn=trn.drop(['due_balance_6'], axis=1)
trn=trn.drop(['due_balance_7'], axis=1)
trn=trn.drop(['due_balance_8'], axis=1)

In [13]:
trn[350:400]

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,mob,open_closed_flag,ever_delinquent_flg,...,writeoff_type_settlement,writeoff_type_fraud_other,writeoff_type_repo,writeoff_type_null,writeoff_date,snapshot,mth_code,industry,overdue_months,overdue_amount
350,1.0,R,1,0,I 13-24,1.0,761+,22,1.0,0.0,...,0,0,0,1,NaN,201801,201908,B,0.0,0.00
351,1.0,R,0,0,A,1.0,621-640,135,1.0,0.0,...,0,0,0,1,NaN,201801,201901,B,0.0,0.00
352,0.0,P,0,0,A,0.0,721-740,21,1.0,0.0,...,0,0,0,1,NaN,201801,201807,C,0.0,0.00
353,1.0,D,0,0,A,1.0,741-760,154,1.0,0.0,...,0,0,0,1,NaN,201801,201809,C,0.0,0.00
354,1.0,I,0,0,A,1.0,761+,28,1.0,0.0,...,0,0,0,1,NaN,201801,201802,A,0.0,0.00
355,1.0,T,0,1,I 0-6,1.0,561-580,98,1.0,0.0,...,0,0,0,1,NaN,201801,201804,B,0.0,0.00
356,0.0,P,0,1,A,1.0,761+,10,1.0,0.0,...,0,0,0,1,NaN,201801,201910,A,0.0,0.00
357,1.0,I,0,0,A,1.0,761+,44,1.0,0.0,...,0,0,0,1,NaN,201801,201809,B,2.0,0.00
358,1.0,I,1,1,A,1.0,681-700,34,1.0,0.0,...,0,0,0,1,NaN,201801,201806,B,0.0,0.00
359,1.0,I,0,1,A,1.0,741-760,35,1.0,0.0,...,0,0,0,1,NaN,201801,201807,C,0.0,0.00


`writeoff_date` is NaN, drop this column.

In [14]:
trn=trn.drop(['writeoff_date'], axis=1)

Create a new column based on the write_off columns:

In [15]:
# Create a new DataFrame containing only the writeoff_type columns
writeoff_type_columns = ['writeoff_type_bko', 'writeoff_type_fraud_kiting', 'writeoff_type_fraud_synthetic', 'writeoff_type_deceased', 'writeoff_type_other', 'writeoff_type_aged',
                         'writeoff_type_settlement', 'writeoff_type_fraud_other', 'writeoff_type_repo', 'writeoff_type_null']
writeoff_types_df = trn[writeoff_type_columns]

# Create a new column 'writeoff_category' containing the name of the column that has the value of 1
trn['writeoff_category'] = writeoff_types_df.idxmax(axis=1)

# Check the result
print(trn[['writeoff_type_null', 'writeoff_category']])

         writeoff_type_null   writeoff_category
0                         1  writeoff_type_null
1                         1  writeoff_type_null
2                         1  writeoff_type_null
3                         1  writeoff_type_null
4                         1  writeoff_type_null
...                     ...                 ...
5758218                   1  writeoff_type_null
5758219                   1  writeoff_type_null
5758220                   1  writeoff_type_null
5758221                   1  writeoff_type_null
5758222                   1  writeoff_type_null

[5758223 rows x 2 columns]


In [16]:
trn=trn.drop(writeoff_type_columns, axis=1)

Drop this following column because there is a large porprotion of 'NaN' values:

In [17]:
trn=trn.drop('variable_rate_margin', axis=1)

In [18]:
trn.head()

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,mob,open_closed_flag,ever_delinquent_flg,...,fraud_writeoff_amt,other_writeoff_amt,promo_bal_amt,recovery_amt,snapshot,mth_code,industry,overdue_months,overdue_amount,writeoff_category
0,1.0,P,0,0,I 0-6,1.0,581-600,3,1.0,0.0,...,0.0,0.0,0.00,0.0,201801,201807,B,0.0,0.0,writeoff_type_null
1,0.0,P,0,0,I 0-6,1.0,721-740,159,1.0,0.0,...,0.0,0.0,1557.24,0.0,201801,201906,C,0.0,0.0,writeoff_type_null
2,1.0,T,0,0,I 7-12,1.0,761+,22,1.0,0.0,...,0.0,0.0,0.00,0.0,201801,201905,A,0.0,0.0,writeoff_type_null
3,1.0,T,0,1,I 0-6,1.0,761+,16,1.0,1.0,...,0.0,0.0,0.00,0.0,201801,201806,B,2.0,0.0,writeoff_type_null
4,1.0,P,0,0,A,0.0,721-740,3,1.0,0.0,...,0.0,0.0,960.00,0.0,201801,201902,B,0.0,0.0,writeoff_type_null


Retrive the column names:

In [19]:
columns = trn.columns
print(columns)

Index(['financial_active', 'net_payment_behaviour_tripd', 'promotion_flag',
       'variable_rate_index', 'account_status_code', 'active_12_mths',
       'bank_fico_buckets_20', 'mob', 'open_closed_flag',
       'ever_delinquent_flg', 'nbr_mths_due', 'stmt_balance', 'prev_balance',
       'net_sales', 'net_payments', 'purchase_active', 'credit_limit_amt',
       'credit_limit_pa', 'closed', 'active', 'charge_off', 'charge_off_aged',
       'charge_off_bk', 'principal_amt', 'principal_amt_chrg_off',
       'total_writeoff_amt', 'fee_chg_off_reversal_amt', 'net_finance_charge',
       'non_principal_amount_gross', 'non_principal_amount_net',
       'non_principal_amount_stmt', 'aged_writeoff_amt',
       'bankruptcy_writeoff_amt', 'fc_reversals', 'fee_reversals',
       'fraud_writeoff_amt', 'other_writeoff_amt', 'promo_bal_amt',
       'recovery_amt', 'snapshot', 'mth_code', 'industry', 'overdue_months',
       'overdue_amount', 'writeoff_category'],
      dtype='object')


Coerce every categorical variables into categorical variables:

In [20]:
categorical_columns = ['financial_active', 'net_payment_behaviour_tripd', 'promotion_flag',
       'variable_rate_index', 'account_status_code', 'active_12_mths',
       'bank_fico_buckets_20', 'open_closed_flag', 'ever_delinquent_flg', 'closed', 'active', 'charge_off', 'charge_off_aged',
       'charge_off_bk', 'purchase_active', 'industry', 'writeoff_category']
response_column = ['principal_amt_chrg_off']
trn[categorical_columns]=trn[categorical_columns].astype('category')

Check the correlations between variables:

In [21]:
trn.corr()

,mob,nbr_mths_due,stmt_balance,prev_balance,net_sales,net_payments,credit_limit_amt,credit_limit_pa,principal_amt,principal_amt_chrg_off,...,fc_reversals,fee_reversals,fraud_writeoff_amt,other_writeoff_amt,promo_bal_amt,recovery_amt,snapshot,mth_code,overdue_months,overdue_amount
mob,1.000000,-0.002764,0.001318,-0.000819,0.000731,0.000220,-0.000255,0.000388,0.000420,-0.007215,...,-0.006832,-0.009957,-0.000432,-0.002389,-0.000357,-0.001752,-0.006084,-0.004170,-0.000821,0.000112
nbr_mths_due,-0.002764,1.000000,-0.003536,0.009805,-0.001208,-0.002423,-0.011681,-0.003607,-0.003652,0.131717,...,0.109790,0.173411,0.000566,0.045433,-0.002426,0.032108,0.008681,0.004583,0.002083,0.001277
stmt_balance,0.001318,-0.003536,1.000000,-0.000532,-0.000502,0.000206,0.000773,0.000908,0.001209,-0.011741,...,-0.010454,-0.016245,-0.000247,-0.003901,0.000548,-0.002218,-0.001001,-0.000654,0.000132,-0.000461
prev_balance,-0.000819,0.009805,-0.000532,1.000000,0.000115,-0.000104,-0.002879,-0.000686,-0.001011,0.029282,...,0.025011,0.038541,0.000379,0.010003,-0.000792,0.006838,-0.001034,-0.001081,0.000314,-0.000494
net_sales,0.000731,-0.001208,-0.000502,0.000115,1.000000,-0.000049,-0.000178,0.000029,0.000433,-0.005481,...,-0.004583,-0.007239,-0.000128,-0.001734,0.000270,-0.001354,-0.005936,-0.003527,0.000277,0.000362
net_payments,0.000220,-0.002423,0.000206,-0.000104,-0.000049,1.000000,0.001064,0.000472,0.000075,-0.006732,...,-0.005891,-0.009079,-0.000339,-0.002287,-0.000465,-0.001962,-0.005954,-0.003471,-0.001039,0.000455
credit_limit_amt,-0.000255,-0.011681,0.000773,-0.002879,-0.000178,0.001064,1.000000,0.000340,0.001102,-0.034228,...,-0.029965,-0.046093,-0.001037,-0.011671,0.000892,-0.008831,0.004923,0.001978,-0.001073,-0.001030
credit_limit_pa,0.000388,-0.003607,0.000908,-0.000686,0.000029,0.000472,0.000340,1.000000,0.000867,-0.010929,...,-0.009323,-0.014506,-0.000249,-0.003605,-0.000496,-0.002943,-0.016609,-0.009396,0.000428,0.000426
principal_amt,0.000420,-0.003652,0.001209,-0.001011,0.000433,0.000075,0.001102,0.000867,1.000000,-0.011382,...,-0.009768,-0.015302,-0.000437,-0.003783,-0.000184,-0.003243,0.001516,0.000347,0.000165,-0.000178
principal_amt_chrg_off,-0.007215,0.131717,-0.011741,0.029282,-0.005481,-0.006732,-0.034228,-0.010929,-0.011382,1.000000,...,0.321833,0.498169,0.003642,0.122641,-0.009148,0.087963,-0.002707,-0.000781,0.005033,0.002364


Builds linear regression model:

In [22]:
columns = trn.columns
print(columns)

Index(['financial_active', 'net_payment_behaviour_tripd', 'promotion_flag',
       'variable_rate_index', 'account_status_code', 'active_12_mths',
       'bank_fico_buckets_20', 'mob', 'open_closed_flag',
       'ever_delinquent_flg', 'nbr_mths_due', 'stmt_balance', 'prev_balance',
       'net_sales', 'net_payments', 'purchase_active', 'credit_limit_amt',
       'credit_limit_pa', 'closed', 'active', 'charge_off', 'charge_off_aged',
       'charge_off_bk', 'principal_amt', 'principal_amt_chrg_off',
       'total_writeoff_amt', 'fee_chg_off_reversal_amt', 'net_finance_charge',
       'non_principal_amount_gross', 'non_principal_amount_net',
       'non_principal_amount_stmt', 'aged_writeoff_amt',
       'bankruptcy_writeoff_amt', 'fc_reversals', 'fee_reversals',
       'fraud_writeoff_amt', 'other_writeoff_amt', 'promo_bal_amt',
       'recovery_amt', 'snapshot', 'mth_code', 'industry', 'overdue_months',
       'overdue_amount', 'writeoff_category'],
      dtype='object')


In [92]:
x=trn[['financial_active', 'total_writeoff_amt', 'fee_chg_off_reversal_amt']]
y=trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.229
Model:                                OLS   Adj. R-squared (uncentered):              0.229
Method:                     Least Squares   F-statistic:                          5.693e+05
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            02:07:18   Log-Likelihood:                     -3.5640e+07
No. Observations:                 5758223   AIC:                                  7.128e+07
Df Residuals:                     5758220   BIC:                                  7.128e+07
Df Model:                               3                                                  
Covariance Type:                nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025

In [84]:
macro_recent=macro_data[223:247]
macro_recent.shape
macro_recent.head()

,Mnemonic,M_FHOFHOPIPOQ.IUSA,M_FYLE.IUSA,M_FRT.IUSA,M_FYPSAVQ.IUSA,M_FC.IUSA,FYPCPICA_FEDB.IUSA,M_FGDP.IUSA,M_FYHHAVGQ.IUSA,M_FYFMMEDQ.IUSA,...,M_FBCFCOFCCQ.IUSA,M_FETP.IUSA,M_FLTV.IUSA,M_FC$.IUSA,M_FEIAMGTTQ.IUSA,M_FXCPMGASUQ.IUSA,M_FRBBBQ.IUSA,M_FHOWNRQ.IUSA,M_FIFR_FEDS.IUSA,M_FIFR$_FEDS.IUSA
223,1/31/2018,248.873161,10761.294,5181.42,7.1,13628.374,53049.86599,20073.62463,138178.1894,75120.61132,...,3.664279275,125.317,79.5,12775.504,2.6712,253.46,4.058025114,64.18299722,782.9992665,616.3053138
224,2/28/2018,250.8678133,10780.022,5215.572,7.2,13668.318,53233.38442,20141.45839,138578.5727,75283.48447,...,3.65003925,125.672,79.3,12765.158,2.70475,256.9375,4.206673162,64.29943946,783.2181714,611.3780129
225,3/31/2018,251.6692577,10808.394,5198.868,7.2,13735.355,53417.1033,20251.37579,139015.1391,75437.82054,...,3.635682638,125.85,79.2,12808.044,2.709,255.9775,4.374298511,64.41750914,784.8495621,608.5116733
226,4/30/2018,252.3962837,10850.006,5220.732,7.2,13792.129,53607.57795,20361.7524,139415.5111,75597.31004,...,3.627843586,125.977,79.2,12863.202,2.8732,272.104,4.530713379,64.50872796,787.5053156,607.2579171
227,5/31/2018,253.7272947,10884.605,5277.108,7.2,13860.171,53798.93234,20482.75073,139951.8411,75770.1069,...,3.634637848,126.238,78.9,12918.212,2.98675,290.12,4.625637301,64.52811155,789.9358207,606.4957157


We choose `Income: Personal - Saving Rate, (%, SA)`, `Per Capita Income`, `Gross Domestic Product`, `Unemployment Rate, (%, SA)`, and `Gross Domestic Product` as five of the variables in the macro dataset because of their high level of relevence.

Firstly, we extract a dataframe to just include the four variables in the first few columns in the macro dataset and only the rows from year 2018 and 2019 (24 months, 24 rows).

In [85]:
select=['Mnemonic','M_FYPSAVQ.IUSA', 'FYPCPICA_FEDB.IUSA', 'M_FGDP.IUSA', 'M_FLBR.IUSA', 'M_FGDP$.IUSA']
macro_recent=macro_recent[select]
macro_recent.head()

,Mnemonic,M_FYPSAVQ.IUSA,FYPCPICA_FEDB.IUSA,M_FGDP.IUSA,M_FLBR.IUSA,M_FGDP$.IUSA
223,1/31/2018,7.1,53049.86599,20073.62463,4,18401.87663
224,2/28/2018,7.2,53233.38442,20141.45839,4.1,18436.77838
225,3/31/2018,7.2,53417.1033,20251.37579,4,18472.72336
226,4/30/2018,7.2,53607.57795,20361.7524,4,18514.26033
227,5/31/2018,7.2,53798.93234,20482.75073,3.8,18563.3238


In [86]:
# date=201801
# for x in range(1, 13) :
#     macro_recent[x,1]=date
#     date=date+1
# date_19=201901
# for y in range(1, 13) :
#     y_c=y+11
#     macro_recent[x,1]=date_19
#     date_19=date_19+1
# macro_recent.head()

macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("1/31/2018", 201801)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("2/28/2018", 201802)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("3/31/2018", 201803)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("4/30/2018", 201804)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("5/31/2018", 201805)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("6/30/2018", 201806)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("7/31/2018", 201807)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("8/31/2018", 201808)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("9/30/2018", 201809)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("10/31/2018", 201810)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("11/30/2018", 201811)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("12/31/2018", 201812)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("1/31/2019", 201901)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("2/28/2019", 201902)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("3/31/2019", 201903)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("4/30/2019", 201904)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("5/31/2019", 201905)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("6/30/2019", 201906)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("7/31/2019", 201907)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("8/31/2019", 201908)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("9/30/2019", 201909)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("10/31/2019", 201910)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("11/30/2019", 201911)
macro_recent['Mnemonic'] = macro_recent['Mnemonic'].replace("12/31/2019", 201912)

macro_recent.tail()

,Mnemonic,M_FYPSAVQ.IUSA,FYPCPICA_FEDB.IUSA,M_FGDP.IUSA,M_FLBR.IUSA,M_FGDP$.IUSA
242,201908,8.7,56918.9553,21548.02532,3.7,19150.28651
243,201909,8.9,57154.14298,21565.45763,3.5,19129.92383
244,201910,8.9,57397.55421,21597.53444,3.6,19112.85692
245,201911,8.9,57651.0369,21683.88241,3.6,19182.00586
246,201912,8.3,57916.66026,21838.1919,3.6,19352.21806


Merge the selected dataset from micro_data and the training dataset

In [113]:
merged_trn = pd.merge(trn, macro_recent, left_on='mth_code', right_on='Mnemonic', how='left')
merged_trn.head()

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,mob,open_closed_flag,ever_delinquent_flg,...,industry,overdue_months,overdue_amount,writeoff_category,Mnemonic,M_FYPSAVQ.IUSA,FYPCPICA_FEDB.IUSA,M_FGDP.IUSA,M_FLBR.IUSA,M_FGDP$.IUSA
0,1.0,P,0,0,I 0-6,1.0,581-600,3,1.0,0.0,...,B,0.0,0.0,writeoff_type_null,201807,7.5,54185.6046,20646.54348,3.8,18672.253
1,0.0,P,0,0,I 0-6,1.0,721-740,159,1.0,0.0,...,C,0.0,0.0,writeoff_type_null,201906,8.5,56461.70244,21361.9346,3.6,19033.3634
2,1.0,T,0,0,I 7-12,1.0,761+,22,1.0,0.0,...,A,0.0,0.0,writeoff_type_null,201905,8.6,56240.50793,21263.8241,3.6,18952.96143
3,1.0,T,0,1,I 0-6,1.0,761+,16,1.0,1.0,...,B,2.0,0.0,writeoff_type_null,201806,7.4,53991.49272,20566.09014,4,18619.50574
4,1.0,P,0,0,A,0.0,721-740,3,1.0,0.0,...,B,0.0,0.0,writeoff_type_null,201902,9.5,55602.13206,20989.81927,3.8,18838.71244


Do model selection:

Firstly, convert the data coming from macro_data into float datatype

In [114]:
merged_trn[select]=merged_trn[select].astype('float')

Get dummy vavirables for the categorical variables in the dataset:

In [115]:
#merged_trn = pd.get_dummies(merged_trn, columns=['financial_active'], drop_first=True)
for cat in range(len(categorical_columns)):
    merged_trn = pd.get_dummies(merged_trn, columns=[categorical_columns[cat]], drop_first=True) 

In [186]:
merged_trn.head()
merged_trn.columns[50:100]

Index(['active_12_mths_1.0', 'bank_fico_buckets_20_581-600',
       'bank_fico_buckets_20_601-620', 'bank_fico_buckets_20_621-640',
       'bank_fico_buckets_20_641-660', 'bank_fico_buckets_20_661-680',
       'bank_fico_buckets_20_681-700', 'bank_fico_buckets_20_701-720',
       'bank_fico_buckets_20_721-740', 'bank_fico_buckets_20_741-760',
       'bank_fico_buckets_20_761+', 'bank_fico_buckets_20_<= 560',
       'bank_fico_buckets_20_Exception', 'bank_fico_buckets_20_Missing',
       'open_closed_flag_1.0', 'ever_delinquent_flg_1.0', 'closed_1',
       'active_1', 'charge_off_1', 'charge_off_aged_1', 'charge_off_bk_1',
       'purchase_active_1.0', 'industry_B', 'industry_C',
       'writeoff_category_writeoff_type_bko',
       'writeoff_category_writeoff_type_deceased',
       'writeoff_category_writeoff_type_fraud_kiting',
       'writeoff_category_writeoff_type_fraud_other',
       'writeoff_category_writeoff_type_null',
       'writeoff_category_writeoff_type_settlement'],
     

Forward Selection of the Linear Regression Model:

In [124]:
x=merged_trn[['mob']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.000
Model:                                OLS   Adj. R-squared (uncentered):              0.000
Method:                     Least Squares   F-statistic:                              948.2
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                   3.46e-208
Time:                            02:55:25   Log-Likelihood:                     -3.6387e+07
No. Observations:                 5758223   AIC:                                  7.277e+07
Df Residuals:                     5758222   BIC:                                  7.277e+07
Df Model:                               1                                                  
Covariance Type:                nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-

In [125]:
x=merged_trn[['mob', 'nbr_mths_due']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.018
Model:                                OLS   Adj. R-squared (uncentered):              0.018
Method:                     Least Squares   F-statistic:                          5.384e+04
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            02:55:38   Log-Likelihood:                     -3.6335e+07
No. Observations:                 5758223   AIC:                                  7.267e+07
Df Residuals:                     5758221   BIC:                                  7.267e+07
Df Model:                               2                                                  
Covariance Type:                nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]

In [126]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.018
Model:                                OLS   Adj. R-squared (uncentered):              0.018
Method:                     Least Squares   F-statistic:                          3.600e+04
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            02:57:49   Log-Likelihood:                     -3.6334e+07
No. Observations:                 5758223   AIC:                                  7.267e+07
Df Residuals:                     5758220   BIC:                                  7.267e+07
Df Model:                               3                                                  
Covariance Type:                nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]

In [127]:
x=merged_trn[['mob', 'nbr_mths_due','stmt_balance','prev_balance']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.019
Model:                                OLS   Adj. R-squared (uncentered):              0.019
Method:                     Least Squares   F-statistic:                          2.839e+04
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            02:58:37   Log-Likelihood:                     -3.6332e+07
No. Observations:                 5758223   AIC:                                  7.266e+07
Df Residuals:                     5758219   BIC:                                  7.266e+07
Df Model:                               4                                                  
Covariance Type:                nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]

In [128]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance', 'prev_balance', 'net_sales']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.019
Model:                                OLS   Adj. R-squared (uncentered):              0.019
Method:                     Least Squares   F-statistic:                          2.274e+04
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            02:59:03   Log-Likelihood:                     -3.6332e+07
No. Observations:                 5758223   AIC:                                  7.266e+07
Df Residuals:                     5758218   BIC:                                  7.266e+07
Df Model:                               5                                                  
Covariance Type:                nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]

In [129]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance', 'prev_balance', 'net_sales', 'net_payments']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.019
Model:                                OLS   Adj. R-squared (uncentered):              0.019
Method:                     Least Squares   F-statistic:                          1.899e+04
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            02:59:27   Log-Likelihood:                     -3.6332e+07
No. Observations:                 5758223   AIC:                                  7.266e+07
Df Residuals:                     5758217   BIC:                                  7.266e+07
Df Model:                               6                                                  
Covariance Type:                nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]

In [130]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance', 'prev_balance', 'net_sales', 'net_payments', 'credit_limit_amt']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.020
Model:                                OLS   Adj. R-squared (uncentered):              0.020
Method:                     Least Squares   F-statistic:                          1.670e+04
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            03:00:05   Log-Likelihood:                     -3.6330e+07
No. Observations:                 5758223   AIC:                                  7.266e+07
Df Residuals:                     5758216   BIC:                                  7.266e+07
Df Model:                               7                                                  
Covariance Type:                nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.

In [131]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance', 'prev_balance', 'net_sales', 'net_payments', 'credit_limit_amt', 'credit_limit_pa']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.020
Model:                                OLS   Adj. R-squared (uncentered):              0.020
Method:                     Least Squares   F-statistic:                          1.464e+04
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            03:00:53   Log-Likelihood:                     -3.6330e+07
No. Observations:                 5758223   AIC:                                  7.266e+07
Df Residuals:                     5758215   BIC:                                  7.266e+07
Df Model:                               8                                                  
Covariance Type:                nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.

In [132]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance', 'prev_balance', 'net_sales', 'net_payments',
            'credit_limit_amt', 'credit_limit_pa', 'principal_amt']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.020
Model:                                OLS   Adj. R-squared (uncentered):              0.020
Method:                     Least Squares   F-statistic:                          1.303e+04
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            03:02:53   Log-Likelihood:                     -3.6330e+07
No. Observations:                 5758223   AIC:                                  7.266e+07
Df Residuals:                     5758214   BIC:                                  7.266e+07
Df Model:                               9                                                  
Covariance Type:                nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.

In [133]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance', 'prev_balance', 'net_sales', 'net_payments',
            'credit_limit_amt', 'credit_limit_pa', 'principal_amt', 'principal_amt_chrg_off']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   1.000
Model:                                OLS   Adj. R-squared (uncentered):              1.000
Method:                     Least Squares   F-statistic:                          2.366e+33
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            03:03:55   Log-Likelihood:                      1.4667e+08
No. Observations:                 5758223   AIC:                                 -2.933e+08
Df Residuals:                     5758213   BIC:                                 -2.933e+08
Df Model:                              10                                                  
Covariance Type:                nonrobust                                                  
                             coef    std err          t      P>|t|      [0.025  

In [134]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance', 'prev_balance', 'net_sales', 'net_payments',
            'credit_limit_amt', 'credit_limit_pa', 'principal_amt', 'principal_amt_chrg_off', 'total_writeoff_amt']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   1.000
Model:                                OLS   Adj. R-squared (uncentered):              1.000
Method:                     Least Squares   F-statistic:                          1.989e+33
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            03:04:19   Log-Likelihood:                      1.4645e+08
No. Observations:                 5758223   AIC:                                 -2.929e+08
Df Residuals:                     5758212   BIC:                                 -2.929e+08
Df Model:                              11                                                  
Covariance Type:                nonrobust                                                  
                             coef    std err          t      P>|t|      [0.025  

In [135]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance', 'prev_balance', 'net_sales', 'net_payments',
            'credit_limit_amt', 'credit_limit_pa', 'principal_amt', 'principal_amt_chrg_off', 'total_writeoff_amt' ,'fee_chg_off_reversal_amt']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   1.000
Model:                                OLS   Adj. R-squared (uncentered):              1.000
Method:                     Least Squares   F-statistic:                          2.042e+33
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            03:05:18   Log-Likelihood:                      1.4678e+08
No. Observations:                 5758223   AIC:                                 -2.935e+08
Df Residuals:                     5758211   BIC:                                 -2.935e+08
Df Model:                              12                                                  
Covariance Type:                nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025

In [136]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance', 'prev_balance', 'net_sales', 'net_payments',
            'credit_limit_amt', 'credit_limit_pa', 'principal_amt', 'principal_amt_chrg_off', 'total_writeoff_amt' ,'fee_chg_off_reversal_amt']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   1.000
Model:                                OLS   Adj. R-squared (uncentered):              1.000
Method:                     Least Squares   F-statistic:                          2.042e+33
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            03:07:54   Log-Likelihood:                      1.4678e+08
No. Observations:                 5758223   AIC:                                 -2.935e+08
Df Residuals:                     5758211   BIC:                                 -2.935e+08
Df Model:                              12                                                  
Covariance Type:                nonrobust                                                  
                               coef    std err          t      P>|t|      [0.025

By this point in the forward selection process, we check that there is not big of a change when we try to add on variables as we go, so we rather just select the variables that we precieve to be effective to the prediction of the response variable.

In [156]:
merged_trn.columns[50:90]

Index(['active_12_mths_1.0', 'bank_fico_buckets_20_581-600',
       'bank_fico_buckets_20_601-620', 'bank_fico_buckets_20_621-640',
       'bank_fico_buckets_20_641-660', 'bank_fico_buckets_20_661-680',
       'bank_fico_buckets_20_681-700', 'bank_fico_buckets_20_701-720',
       'bank_fico_buckets_20_721-740', 'bank_fico_buckets_20_741-760',
       'bank_fico_buckets_20_761+', 'bank_fico_buckets_20_<= 560',
       'bank_fico_buckets_20_Exception', 'bank_fico_buckets_20_Missing',
       'open_closed_flag_1.0', 'ever_delinquent_flg_1.0', 'closed_1',
       'active_1', 'charge_off_1', 'charge_off_aged_1', 'charge_off_bk_1',
       'purchase_active_1.0', 'industry_B', 'industry_C',
       'writeoff_category_writeoff_type_bko',
       'writeoff_category_writeoff_type_deceased',
       'writeoff_category_writeoff_type_fraud_kiting',
       'writeoff_category_writeoff_type_fraud_other',
       'writeoff_category_writeoff_type_null',
       'writeoff_category_writeoff_type_settlement'],
     

In [157]:
x=merged_trn[['mob', 'nbr_mths_due', 'stmt_balance','net_sales',
       'net_payments', 'credit_limit_amt', 'principal_amt', 'promo_bal_amt',
       'recovery_amt', 'active_12_mths_1.0', 'charge_off_1',
       'charge_off_aged_1', 'charge_off_bk_1', 'purchase_active_1.0',
       'Mnemonic', 'M_FYPSAVQ.IUSA', 'FYPCPICA_FEDB.IUSA', 'M_FGDP.IUSA', 'M_FLBR.IUSA', 'M_FGDP$.IUSA']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                              OLS Regression Results                              
Dep. Variable:     principal_amt_chrg_off   R-squared:                       0.368
Model:                                OLS   Adj. R-squared:                  0.368
Method:                     Least Squares   F-statistic:                 1.764e+05
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):               0.00
Time:                            03:31:07   Log-Likelihood:            -3.5064e+07
No. Observations:                 5758223   AIC:                         7.013e+07
Df Residuals:                     5758203   BIC:                         7.013e+07
Df Model:                              19                                         
Covariance Type:                nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [158]:
curr_col= ['mob', 'nbr_mths_due', 'stmt_balance','net_sales',
       'net_payments', 'credit_limit_amt', 'principal_amt', 'promo_bal_amt',
       'recovery_amt', 'active_12_mths_1.0', 'charge_off_1',
       'charge_off_aged_1', 'charge_off_bk_1', 'purchase_active_1.0',
       'Mnemonic', 'M_FYPSAVQ.IUSA', 'FYPCPICA_FEDB.IUSA', 'M_FGDP.IUSA', 'M_FLBR.IUSA', 'M_FGDP$.IUSA']
print(results.pvalues[curr_col])

mob                    5.468846e-01
nbr_mths_due           8.802780e-24
stmt_balance           5.203074e-01
net_sales              7.673833e-01
net_payments           9.912945e-01
credit_limit_amt       9.300643e-01
principal_amt          8.378183e-01
promo_bal_amt          9.882093e-01
recovery_amt           2.332560e-86
active_12_mths_1.0     5.098911e-01
charge_off_1           0.000000e+00
charge_off_aged_1      3.121147e-80
charge_off_bk_1        3.976851e-42
purchase_active_1.0    8.298087e-01
Mnemonic               6.156361e-01
M_FYPSAVQ.IUSA         4.069216e-01
FYPCPICA_FEDB.IUSA     5.622233e-01
M_FGDP.IUSA            5.099002e-01
M_FLBR.IUSA            3.425521e-01
M_FGDP$.IUSA           8.771046e-01
dtype: float64


Drop the variables that has the p-value > 0.05, and this is our final model:

In [195]:
x=merged_trn[['nbr_mths_due',
       'recovery_amt']]
y=merged_trn['principal_amt_chrg_off']
lm=sm.OLS(y,x)
results=lm.fit()
print(results.summary())

                                   OLS Regression Results                                  
Dep. Variable:     principal_amt_chrg_off   R-squared (uncentered):                   0.025
Model:                                OLS   Adj. R-squared (uncentered):              0.025
Method:                     Least Squares   F-statistic:                          7.494e+04
Date:                    Sun, 26 Mar 2023   Prob (F-statistic):                        0.00
Time:                            04:12:29   Log-Likelihood:                     -3.6314e+07
No. Observations:                 5758223   AIC:                                  7.263e+07
Df Residuals:                     5758221   BIC:                                  7.263e+07
Df Model:                               2                                                  
Covariance Type:                nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]

In [196]:
lm=sm.OLS(y,x).fit()
forecast_data = pd.read_csv('forecast_starting_data.csv')
forecast_data=forecast_data[['nbr_mths_due', 'recovery_amt', 'charge_off', 'charge_off_aged', 'charge_off_bk']]
forecast_data = pd.get_dummies(forecast_data, columns=['charge_off', 'charge_off_aged', 'charge_off_bk'], drop_first=True)
#results.predict(forecast_data)

In [197]:
forecast_data.head()

,nbr_mths_due,recovery_amt
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [199]:
result=results.predict(forecast_data)
result[0:50]

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
5      0.000000
6      0.000000
7      0.000000
8      0.000000
9      0.000000
10     0.000000
11     0.000000
12     0.000000
13     0.000000
14    64.239077
15     0.000000
16     0.000000
17     0.000000
18     0.000000
19     0.000000
20     0.000000
21     0.000000
22     0.000000
23     0.000000
24     0.000000
25     0.000000
26     0.000000
27     0.000000
28     0.000000
29     0.000000
30     0.000000
31     0.000000
32     0.000000
33     0.000000
34     0.000000
35     0.000000
36     0.000000
37     0.000000
38     0.000000
39     0.000000
40     0.000000
41     0.000000
42     0.000000
43     0.000000
44     0.000000
45     0.000000
46     0.000000
47     0.000000
48     0.000000
49     0.000000
dtype: float64

In [1]:
result.scatterplot()

NameError: name 'result' is not defined